In [27]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")

import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np
from geojson.geometry import Point, Polygon, MultiLineString

client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response
import geopandas as gpd
from shapely import geometry
from shapely.geometry import Point

In [107]:
mq_hr = pd.read_pickle('../cgarrido/berlin_data/traffic/mq_hr_with_location.pkl')

In [108]:
mq_hr

,mq_name,tag,stunde,qualitaet,q_kfz_mq_hr,v_kfz_mq_hr,q_pkw_mq_hr,v_pkw_mq_hr,q_lkw_mq_hr,v_lkw_mq_hr,lat,lon
0,TE004,04.01.2022,10,1.0,518,42,501,42,17,41,13.261301,52.436642
1,TE004,04.01.2022,11,1.0,610,40,596,40,14,42,13.261301,52.436642
2,TE004,04.01.2022,12,1.0,628,36,609,36,19,36,13.261301,52.436642
3,TE004,04.01.2022,13,1.0,624,41,598,41,26,38,13.261301,52.436642
4,TE004,04.01.2022,14,1.0,656,42,628,42,28,42,13.261301,52.436642
...,...,...,...,...,...,...,...,...,...,...,...,...
114673,TE559,31.01.2022,19,1.0,186,29,186,29,0,-1,13.387089,52.531362
114674,TE559,31.01.2022,20,1.0,128,28,128,28,0,-1,13.387089,52.531362
114675,TE559,31.01.2022,21,1.0,99,33,99,33,0,-1,13.387089,52.531362
114676,TE559,31.01.2022,22,1.0,101,34,100,34,1,26,13.387089,52.531362


In [109]:
ber_hx = pd.read_pickle('../cgarrido/berlin_data/hex_grid/hexagons.pkl')

In [110]:
ber_hx

,hexagon_id,hexagon_x,hexagon_y,geometry,lat,lon
0,0,0,0,"POLYGON ((1490671.563 6896601.483, 1490769.010...",52.525749,13.409063
1,1,1,1,"POLYGON ((1486732.502 6896205.302, 1486830.610...",52.523589,13.373677
2,2,0,1,"POLYGON ((1488848.594 6893262.031, 1488946.321...",52.507496,13.392682
3,3,1,0,"POLYGON ((1488555.212 6899545.594, 1488653.039...",52.541839,13.390056
4,4,-1,-1,"POLYGON ((1494611.363 6896995.788, 1494708.149...",52.527898,13.444455
...,...,...,...,...,...,...
266,266,-9,-7,"POLYGON ((1522486.772 6893415.233, 1522578.827...",52.508285,13.694856
267,267,-9,-9,"POLYGON ((1526156.027 6900082.526, 1526247.513...",52.544718,13.727827
268,268,-7,-9,"POLYGON ((1521933.780 6905992.816, 1522026.023...",52.577001,13.689907
269,269,-8,-9,"POLYGON ((1524045.347 6903037.871, 1524137.212...",52.560864,13.708871


In [191]:
from pyproj import Transformer

#transformer = Transformer.from_crs("EPSG:25833", "EPSG:4326" , always_xy=True) # transform geometry to lat/lon

transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")
#from pyproj import Transformer
#transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326") # transform geometry to lat/lon
#transformer = Transformer.from_crs('EPSG:25833', 'EPSG:4326', always_xy=True)
#import math

#def convert_to_lat_lon(x, y, stn_id):
#    if math.isnan(x):
#        return transformer.transform(locations[stn_id][0], locations[stn_id][1])
#    else :
#        return transformer.transform(x, y)

In [192]:
mq_hr['x'], mq_hr['y'] = transformer.transform(mq_hr.lat,mq_hr.lon,direction= 'INVERSE')

In [171]:
mq_hr

,mq_name,tag,stunde,qualitaet,q_kfz_mq_hr,v_kfz_mq_hr,q_pkw_mq_hr,v_pkw_mq_hr,q_lkw_mq_hr,v_lkw_mq_hr,lat,lon,geometry,x,y
0,TE004,04.01.2022,10,1.0,518,42,501,42,17,41,13.261301,52.436642,POINT (0.00047 0.00012),5.837220e+06,1.489601e+06
1,TE004,04.01.2022,11,1.0,610,40,596,40,14,42,13.261301,52.436642,POINT (0.00047 0.00012),5.837220e+06,1.489601e+06
2,TE004,04.01.2022,12,1.0,628,36,609,36,19,36,13.261301,52.436642,POINT (0.00047 0.00012),5.837220e+06,1.489601e+06
3,TE004,04.01.2022,13,1.0,624,41,598,41,26,38,13.261301,52.436642,POINT (0.00047 0.00012),5.837220e+06,1.489601e+06
4,TE004,04.01.2022,14,1.0,656,42,628,42,28,42,13.261301,52.436642,POINT (0.00047 0.00012),5.837220e+06,1.489601e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114673,TE559,31.01.2022,19,1.0,186,29,186,29,0,-1,13.387089,52.531362,POINT (0.00047 0.00012),5.847764e+06,1.503991e+06
114674,TE559,31.01.2022,20,1.0,128,28,128,28,0,-1,13.387089,52.531362,POINT (0.00047 0.00012),5.847764e+06,1.503991e+06
114675,TE559,31.01.2022,21,1.0,99,33,99,33,0,-1,13.387089,52.531362,POINT (0.00047 0.00012),5.847764e+06,1.503991e+06
114676,TE559,31.01.2022,22,1.0,101,34,100,34,1,26,13.387089,52.531362,POINT (0.00047 0.00012),5.847764e+06,1.503991e+06


In [193]:
#s = gpd.GeoSeries([Point(x,y) for x,y in zip(mq_hr.x, mq_hr.y)])
#gpd_df = gpd.GeoDataFrame(mq_hr, geometry=s, crs='epsg:25833')

#df_2export = gpd.GeoDataFrame(df_2export, columns = df_2export.columns, geometry=gpd.points_from_xy(df_2export.longitud, df_2export.latitud), crs='epsg:4326')

gpd_df = gpd.GeoDataFrame(
    mq_hr, columns = mq_hr.columns, geometry=gpd.points_from_xy(mq_hr.y, mq_hr.x), crs="epsg:3857"
)

s = gpd.GeoSeries([Point(x,y) for x, y in zip(mq_hr['x'], mq_hr['y'])])
gpd_df = gpd.GeoDataFrame(mq_hr, geometry=s, crs='epsg:3857')
    #gpd_df[magnitude] = gpd_df[magnitude].to_crs(epsg=3857)

#gpd_df = gpd.GeoDataFrame(
#    mq_hr, columns = mq_hr.columns, geometry=Point(transformer.transform(52.525749,13.409063,direction= 'INVERSE')), crs="epsg:4326"
#)

In [188]:
gpd_df

,mq_name,tag,stunde,qualitaet,q_kfz_mq_hr,v_kfz_mq_hr,q_pkw_mq_hr,v_pkw_mq_hr,q_lkw_mq_hr,v_lkw_mq_hr,lat,lon,geometry,x,y
0,TE004,04.01.2022,10,1.0,518,42,501,42,17,41,13.261301,52.436642,POINT (4839859.160 1830882.873),4.839859e+06,1.830883e+06
1,TE004,04.01.2022,11,1.0,610,40,596,40,14,42,13.261301,52.436642,POINT (4839859.160 1830882.873),4.839859e+06,1.830883e+06
2,TE004,04.01.2022,12,1.0,628,36,609,36,19,36,13.261301,52.436642,POINT (4839859.160 1830882.873),4.839859e+06,1.830883e+06
3,TE004,04.01.2022,13,1.0,624,41,598,41,26,38,13.261301,52.436642,POINT (4839859.160 1830882.873),4.839859e+06,1.830883e+06
4,TE004,04.01.2022,14,1.0,656,42,628,42,28,42,13.261301,52.436642,POINT (4839859.160 1830882.873),4.839859e+06,1.830883e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114673,TE559,31.01.2022,19,1.0,186,29,186,29,0,-1,13.387089,52.531362,POINT (4849394.428 1850134.674),4.849394e+06,1.850135e+06
114674,TE559,31.01.2022,20,1.0,128,28,128,28,0,-1,13.387089,52.531362,POINT (4849394.428 1850134.674),4.849394e+06,1.850135e+06
114675,TE559,31.01.2022,21,1.0,99,33,99,33,0,-1,13.387089,52.531362,POINT (4849394.428 1850134.674),4.849394e+06,1.850135e+06
114676,TE559,31.01.2022,22,1.0,101,34,100,34,1,26,13.387089,52.531362,POINT (4849394.428 1850134.674),4.849394e+06,1.850135e+06


In [194]:
gpd_df_with_hex = gpd.sjoin(gpd_df, ber_hx, how='left').drop(columns=['index_right'])
gpd_df_with_hex = gpd_df_with_hex.sort_values(by=['tag','stunde'])
gpd_df_with_hex = gpd_df_with_hex.reset_index(drop=True)

In [195]:
gpd_df_with_hex

,mq_name,tag,stunde,qualitaet,q_kfz_mq_hr,v_kfz_mq_hr,q_pkw_mq_hr,v_pkw_mq_hr,q_lkw_mq_hr,v_lkw_mq_hr,lat_left,lon_left,geometry,x,y,hexagon_id,hexagon_x,hexagon_y,lat_right,lon_right
0,TE005,01.01.2022,1,1.0,307,50,184,51,123,49,13.263106,52.435112,POINT (5837050.014 1489807.510),5.837050e+06,1.489808e+06,NaN,NaN,NaN,NaN,NaN
1,TE009,01.01.2022,1,1.0,197,62,192,62,5,65,13.379267,52.436141,POINT (5837164.470 1503096.115),5.837164e+06,1.503096e+06,NaN,NaN,NaN,NaN,NaN
2,TE010,01.01.2022,1,1.0,147,40,141,40,6,38,13.364413,52.414876,POINT (5834797.325 1501396.514),5.834797e+06,1.501397e+06,NaN,NaN,NaN,NaN,NaN
3,TE011,01.01.2022,1,1.0,165,39,165,39,0,-1,13.359741,52.409598,POINT (5834209.802 1500861.908),5.834210e+06,1.500862e+06,NaN,NaN,NaN,NaN,NaN
4,TE012,01.01.2022,1,1.0,380,55,374,55,6,57,13.400052,52.416157,POINT (5834939.922 1505474.545),5.834940e+06,1.505475e+06,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114673,TE524,31.01.2022,23,1.0,89,50,83,50,6,55,13.458537,52.453576,POINT (5839105.363 1512168.099),5.839105e+06,1.512168e+06,NaN,NaN,NaN,NaN,NaN
114674,TE526,31.01.2022,23,1.0,42,50,39,51,3,44,13.484226,52.424770,POINT (5835898.677 1515108.632),5.835899e+06,1.515109e+06,NaN,NaN,NaN,NaN,NaN
114675,TE530,31.01.2022,23,1.0,259,64,246,64,13,60,13.388168,52.437281,POINT (5837291.385 1504114.562),5.837291e+06,1.504115e+06,NaN,NaN,NaN,NaN,NaN
114676,TE558,31.01.2022,23,1.0,40,32,37,32,0,-1,13.387209,52.531592,POINT (5847790.042 1504004.882),5.847790e+06,1.504005e+06,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pyproj as proj

# setup your projections
crs_wgs = proj.Proj(init='epsg:4326') # assuming you're using WGS84 geographic
crs_bng = proj.Proj(init='epsg:27700') # use a locally appropriate projected CRS

# then cast your geographic coordinate pair to the projected system
x, y = proj.transform(crs_wgs, crs_bng, input_lon, input_lat)

In [69]:
ber_hx

,hexagon_id,hexagon_x,hexagon_y,geometry,lat,lon
0,0,0,0,"POLYGON ((1490671.563 6896601.483, 1490769.010...",52.525749,13.409063
1,1,1,1,"POLYGON ((1486732.502 6896205.302, 1486830.610...",52.523589,13.373677
2,2,0,1,"POLYGON ((1488848.594 6893262.031, 1488946.321...",52.507496,13.392682
3,3,1,0,"POLYGON ((1488555.212 6899545.594, 1488653.039...",52.541839,13.390056
4,4,-1,-1,"POLYGON ((1494611.363 6896995.788, 1494708.149...",52.527898,13.444455
...,...,...,...,...,...,...
266,266,-9,-7,"POLYGON ((1522486.772 6893415.233, 1522578.827...",52.508285,13.694856
267,267,-9,-9,"POLYGON ((1526156.027 6900082.526, 1526247.513...",52.544718,13.727827
268,268,-7,-9,"POLYGON ((1521933.780 6905992.816, 1522026.023...",52.577001,13.689907
269,269,-8,-9,"POLYGON ((1524045.347 6903037.871, 1524137.212...",52.560864,13.708871


In [56]:
from shapely.geometry import Point, Polygon

In [143]:
tmp_pt_2 = Point(transformer.transform(13.409063, 52.525749,direction= 'INVERSE'))

In [176]:
tmp_pt = Point(transformer.transform(52.525749,13.409063,direction= 'INVERSE'))


In [153]:
transformer.transform(52.525749,13.409063,direction= 'INVERSE')

(1492690.0651749251, 6895751.623758236)

In [152]:
print(tmp_pt)

POINT (1492690.0651749251 6895751.623758236)


In [145]:
tmp_pt_2.within(ber_hx.iloc[0]['geometry'])

False

In [177]:
tmp_pt.within(ber_hx.iloc[0]['geometry'])

True

In [96]:
transformer.transform(52.525749,13.409063,direction= 'INVERSE')

(392069.0090710515, 5820705.488962325)

In [97]:
transformer.transform(13.409063, 52.525749, direction= 'INVERSE')

(4848119.300481026, 1852976.4846023808)

In [84]:
gpd.points_from_xy(x=[13.409063], y=[52.525749], crs = 'EPSG:4326')[0].within(ber_hx.iloc[0]['geometry'])

False

In [134]:
u = utm.from_latlon(52.525749, 13.409063)

In [136]:
Point(u[0],u[1]).within(ber_hx.iloc[0]['geometry'])

False

In [85]:
gpd.points_from_xy(x=[13.409063], y=[52.525749], crs = 'EPSG:4326')

<GeometryArray>
[<POINT (13.409 52.526)>]
Length: 1, dtype: geometry

In [86]:
gpd.points_from_xy(x=[13.409063], y=[52.525749], crs = 'EPSG:3857')

<GeometryArray>
[<POINT (13.409 52.526)>]
Length: 1, dtype: geometry

In [87]:
gpd.points_from_xy(x=[13.409063], y=[52.525749], crs = 'EPSG:25833')

<GeometryArray>
[<POINT (13.409 52.526)>]
Length: 1, dtype: geometry

In [117]:
!pip install utm
import utm
u = utm.from_latlon(12.917091, 77.573586)

  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6094 sha256=3def7c7d7b206ab50d6e11567b1e3d0b009d02595853c6e12010c5278766e57c
  Stored in directory: /home/scorpio/.cache/pip/wheels/65/e2/d8/878a8cc986641056fbfebefc4d8eb64238a7b6d3426e86b447
Successfully built utm


ModuleNotFoundError: No module named 'utm'

In [125]:
import utm


In [126]:
u = utm.from_latlon(12.917091, 77.573586)

In [128]:
u

(779260.623156606, 1429369.8665238516, 43, 'P')

In [200]:
for indxe,hexgn in ber_hx.iterrows():
    tmp_pt = Point(transformer.transform(52.436642,13.261301,direction= 'INVERSE'))
    if tmp_pt.within(hexgn['geometry']) == True:
        print('true')
    

true


In [204]:
def find_hexgn(lat,lon):
    for indxe,hexgn in ber_hx.iterrows():
        tmp_pt = Point(transformer.transform(lon,lat,direction= 'INVERSE'))
        if tmp_pt.within(hexgn['geometry']) == True:
            return hexgn['hexagon_id']

In [208]:
mq_hr['hx_id'] = mq_hr.apply(lambda x : find_hexgn(x['lat'],x['lon']), axis=1)

In [216]:
mq_hr.columns

Index(['mq_name', 'tag', 'stunde', 'qualitaet', 'q_kfz_mq_hr', 'v_kfz_mq_hr',
       'q_pkw_mq_hr', 'v_pkw_mq_hr', 'q_lkw_mq_hr', 'v_lkw_mq_hr', 'lat',
       'lon', 'geometry', 'x', 'y', 'hx_id'],
      dtype='object')

In [213]:
mq_hr.to_pickle('../cgarrido/berlin_data/traffic/mq_hr_with_hexagon_ids.pkl')

In [217]:
mq_hr_required = mq_hr[['tag', 'stunde',  'q_kfz_mq_hr', 'v_kfz_mq_hr',
       'q_pkw_mq_hr', 'v_pkw_mq_hr', 'q_lkw_mq_hr', 'v_lkw_mq_hr','hx_id']]

In [220]:
mq_hr_required.groupby(['hx_id','tag','stunde']).mean().reset_index().to_pickle('../cgarrido/berlin_data/traffic/mq_hr_with_hexagon_ids_aggr.pkl')